# Линейная регрессия

Поработаем с датасетом Бостон.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
%pylab inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
print(data.DESCR)

In [ ]:
X_full = data.data
y_full = data.target

## Строим базовую модель

Разбейте данные на train и test. Обучите линейную регрессию на train и сделайте предсказание на train и test.

Выведите MSE и r2 на train и на test.

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X_full, y_full, test_size=0.25, random_state=234)

model = LinearRegression()

model.fit(Xtrain, ytrain) 

pred_train = model.predict(Xtrain)
pred_test = model.predict(Xtest)

In [ ]:
from sklearn.metrics import r2_score

print(r2_score(ytrain, pred_train))
print(r2_score(ytest, pred_test))

Посмотрите на качество линейной регрессии (из sklearn) на кросс-валидации. Измеряйте r2.

In [ ]:
from sklearn.model_selection import cross_val_score

model = LinearRegression()

cross_val_score(model, X_full, y_full, cv=3, scoring='r2').mean()

Как вы думаете, почему качество на кросс-валидации такое низкое?

Будем пробовать улучшать модель.

## Улучшаем базовую модель

Нарисуем матрицу корреляций признаков.

In [ ]:
import seaborn as sns

data1 = pd.DataFrame(data= np.c_[data['data'], data['target']],
                     columns= list(data['feature_names']) + ['target'])

corr = data1.corr()

sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            cmap="YlGnBu")

In [ ]:
corr

Посмотрим на значение коэффициентов корреляции признаков с таргетом.

In [ ]:
data1[data1.columns[1:]].corr()['target'][:-1].sort_values()

In [ ]:
y = data1['target']

X = data1.drop('target', axis=1)

DIS - сильно коррелирует с другими признаками и при этом не коррелирует с таргетом, поэтому пробуем его удалить.

In [ ]:
del X['DIS']

In [ ]:
cross_val_score(LinearRegression(), X, y, cv=3, scoring='r2').mean()

Качество модели значительно возросло, но всё ещё плохое.

## Детальная работа с признаками для улучшения качества

In [ ]:
data1.columns

In [ ]:
for c in data1.columns:
    if c != 'target':
        print(c)
        scatter(data1[c], data1['target'])
        show()

Добавим новый признак

In [ ]:
X['LSTAT_new'] = 1. / X['LSTAT']

cross_val_score(LinearRegression(), X, y, cv=3, scoring='r2').mean()

In [ ]:
scatter(X['LSTAT_new'], y)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25)

model = LinearRegression()

model.fit(Xtrain, ytrain) 

for a,b in zip(X.columns, model.coef_):
    print(a,b)

In [ ]:
np.corrcoef(X['NOX'], X['RM'])[0][1]

In [ ]:
X['NOX*RM'] = X['NOX'] * X['RM']

In [ ]:
cross_val_score(LinearRegression(), X, y, cv=3, scoring='r2').mean()

## Выводы: как улучшить качество модели?

* Первый подход - меняем модель на более сложную.

* Подходы к улучшению качества, не меняя модель:

1) Искать мультиколлинеарность, удалять зависимые признаки. 

2) Пытаться удалить некоррелирующие с таргетом признаки. 

в линейной модели:

3) Смотрим на графики зависимости таргета с каждым признаком в отдельности, и пытаемся брать функции от признаков (квадрат, синус, экспонента...) 

4) Добавляем нелинейные взаимодействия признаков (a*b, a^b ит.д.) -

## Интерпретация модели

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25)

sc = StandardScaler()
sc.fit(Xtrain)

Xtrain = pd.DataFrame(sc.transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
Xtest = pd.DataFrame(sc.transform(Xtest), index=Xtest.index, columns=Xtest.columns)

Xtrain_scaled = sc.transform(Xtrain)
Xtest_scaled = sc.transform(Xtest)

model = LinearRegression()

model.fit(Xtrain_scaled, ytrain) 

In [ ]:
def visualize_coefficients(model, feature_names, n_features=7):

    coef = model.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_features:]
    negative_coefficients = np.argsort(coef)[:n_features]
    all_coefs = np.hstack([negative_coefficients, positive_coefficients])

    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[all_coefs]]
    plt.bar(np.arange(2*n_features), coef[all_coefs], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1+2*n_features), feature_names[all_coefs], rotation=60, ha="right")
    
visualize_coefficients(model, X.columns.values)